In [1]:
import os

#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())
print(os.listdir(os.getcwd()))
# os.chdir("Desktop/iMac_Drive/코스웍/2022 고급기계학습주제 (김광인)/과제/CW2") # Change path

/Users/ming/Desktop/iMac_Drive/코스웍/2022 고급기계학습주제 (김광인)/과제/CW2
['.DS_Store', 'Untitled.ipynb', 'CW2.pdf', 'model', '.ipynb_checkpoints', 'CW2_GP.ipynb', 'data']


In [3]:
from model.gaussian_process import MOGPR, SR_MOGPR
from model.kernel import LinearKernel, GaussianKernel

In [8]:
import pandas as pd
import numpy as np
import time
from scipy.linalg import kron
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
# Fix seed
seed = 1
np.random.seed(seed)

## Data: SARCOS
SARCOS is a Robot arm dataset. It aims to predict 7 Torque columns given 7 Joint positions, 7 Velocities, 7 Accelerations.

In [14]:
train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

X_train, y_train = train_data.iloc[:,:-7], train_data.iloc[:,21:]
# X_train, X_valid, y_train, y_valid = train_test_split(train_data.iloc[:,:-7], train_data.iloc[:,21:], test_size=0.1, random_state=seed)
X_test, y_test = test_data.iloc[:,:-7], test_data.iloc[:,21:]

print("SARCOS data size")
print("Train", X_train.shape, y_train.shape)
# print("Valid", X_valid.shape, y_valid.shape)
print("Test", X_test.shape, y_test.shape)

SARCOS data size
Train (39484, 21) (39484, 7)
Test (5000, 21) (5000, 7)


## Kernel
We use below linear and non-linear kernels:
- Linear inner product kernel
$$ k(x_1,x_2) = \sigma_0 + x_1 \cdot x_2 $$
- Non-linear Gaussian kernel
$$ k(x_1,x_2) = \exp(-{1 \over \sigma_k^2}\left|x_1-x_2\right|^2)$$

# 1. Comparison of Exact MOGPR and Sparse MOGPR
Here, we would see the time complexity of two algorithms. Obviously, __Sparse MOGPR is better.__<br>
Both use __partial datasets__ due to time and memory limitations

In [15]:
pX_train = X_train[:2000]
py_train = y_train[:2000]
pX_test = X_test[:500]
py_test = y_test[:500]

print("partial SARCOS data size")
print("Train", pX_train.shape, py_train.shape)
print("Test", pX_test.shape, py_test.shape)

partial SARCOS data size
Train (2000, 21) (2000, 7)
Test (500, 21) (500, 7)


In [ ]:
full_model = SR_MOGPR(pX_train, py_train, GaussianKernel())

start = time.time()

# My Code
y_predict = full_model.predict(pX_test)

end = time.time()
print("Run time [s]: ",end-start)